In [ ]:
# import sys
# !{sys.executable} -m pip install xgboost

In [ ]:
import xgboost as xgb

#Other imports
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Data import
diamonds = sns.load_dataset("diamonds")
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [ ]:
diamonds.shape

(53940, 10)

In [ ]:
diamonds.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


## Objective

Predict diamond price from their physical characteristics.

In [ ]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

## Building an XGBoost DMatrix

XGBoost has the ability to deal with categorical variables internally, rather than requiring the user to cast each as one-hot encoded values. But, these input categorical features need to be type `category` in Pandas.

In [ ]:
#Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()
cats

['cut', 'color', 'clarity']

In [ ]:
for col in cats:
    X[col] = X[col].astype('category')
X.dtypes #See category types

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
x           float64
y           float64
z           float64
dtype: object

In [ ]:
X, y = diamonds.drop('price', axis=1), diamonds[['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1973)

The `DMatrix` type is the method for storing data in XGBoost that is fast, & highly memory efficient.

In [ ]:
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

## Loss Functions & Metrics in XGBoost

A _loss function_ is used by a machine learning model to minimize the _differences_ between actual ("ground truth") values and model predictions. Separately, a metric is chosen  by the ML engineer to measure the _similarity_ between ground truth and model predictions. Metrics are used after training to evaluate overall performance (e.g., accuracy, recall, precision).

The loss function is specified in a dictionary as the `"objective"` as follows:

In [ ]:
# Define hyperparameters

params = {"objective": "reg:squarederror", "tree_method": 'hist'}#"gpu_hist"}

Inside this initial `params`, we are also setting `tree_method` to `gpu_hist`, which enables GPU acceleration. If you don't have a GPU, you can omit the parameter or set it to `hist`.

## Boosting Rounds

XGBoost's time to shine. The `num_boost_round` parameter sets the number of _boosting rounds_. For the given loss function (RMSE in this example), XGB minimizes the loss in small incremental rounds. In practice, this parameter is found through hyperparameter tuning. But, here we will set it to 100.

In [ ]:
n = 100
model = xgb.train(
    params=params
    , dtrain=dtrain_reg
    , num_boost_round=n
)

## Evaluation

XGBoost modes has a `.predict()` method that we will use to compare to actual diamond price values and calculation an overall RMSE.

In [ ]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 538.167


This performance value is a good baseline. We can see the minimization of error accross boosting rounds by passing the validation set `dtest_reg` to the model as well. Because boosting round will often number in the thousands, we can use the `verbose_eval` parameter to control how often updates are printed:

In [ ]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]
model = xgb.train(
    params=params
    , dtrain=dtrain_reg
    , num_boost_round=n
    , evals=evals
    , verbose_eval=10
)

[0]	train-rmse:3977.45382	validation-rmse:3964.19581
[10]	train-rmse:555.33206	validation-rmse:591.31313
[20]	train-rmse:492.49286	validation-rmse:549.47014
[30]	train-rmse:465.39701	validation-rmse:542.16912
[40]	train-rmse:447.86009	validation-rmse:540.88764
[50]	train-rmse:431.70047	validation-rmse:538.63653
[60]	train-rmse:421.17886	validation-rmse:538.36693
[70]	train-rmse:403.50555	validation-rmse:538.51618
[80]	train-rmse:394.27771	validation-rmse:537.13333
[90]	train-rmse:383.51864	validation-rmse:537.37270
[99]	train-rmse:374.76291	validation-rmse:538.16748



## Early Stopping

Notice above that the loss does not inherently decrease as the number of boosting rounds increases. At a larger scale, let's see what happens at 5000 boosting rounds (verbosity at 500).

In [ ]:
model = xgb.train(
    params=params
    , dtrain=dtrain_reg
    , num_boost_round=5000
    , evals=evals
    , verbose_eval=500
)

[0]	train-rmse:3977.45382	validation-rmse:3964.19581
[500]	train-rmse:200.58807	validation-rmse:552.49024
[1000]	train-rmse:127.96630	validation-rmse:558.97226
[1500]	train-rmse:89.94796	validation-rmse:562.83895
[2000]	train-rmse:67.12858	validation-rmse:565.19271
[2500]	train-rmse:53.08825	validation-rmse:566.63096
[3000]	train-rmse:43.08917	validation-rmse:567.41885
[3500]	train-rmse:36.61729	validation-rmse:568.02380
[4000]	train-rmse:32.02086	validation-rmse:568.44478
[4500]	train-rmse:28.70827	validation-rmse:568.78321
[4999]	train-rmse:26.39354	validation-rmse:568.95641


The training RMSE continues to decline. However, the validation RMSE levels off and appears to be slightly worse than some of the earliest boosting rounds. The model is memorizing the training data rather than generalizing for future observations, which ultimately reduce its performance. Instead we want a model in the _golden middle_. We can find this using XGBoost's early stopping criterea, `early_stopping_rounds = x`, where the model will hald training if the validation performance does not improve for `x` rounds. Setting this parameter allows us to choose as many boosting rounds as we like, such as `num_boost_round=10000`.

In [ ]:
model = xgb.train(
    params=params
    , dtrain=dtrain_reg
    , num_boost_round=10000
    , evals=evals
    , verbose_eval=500
    , early_stopping_rounds=50
)

[0]	train-rmse:3977.45382	validation-rmse:3964.19581
[132]	train-rmse:346.56655	validation-rmse:542.94340


The XGB model stopped itself at an early round as the performance in the validation set plateaued.

## XGBoost Cross-Validation

Using a single test-set to validate the results of a machine learning model is problematic, as the model is able to implicitly memorize the test-set in its iterative optimization steps. This is because the hyperparameter optimization will optimize to the specific validation test-set, which may not generalize as well to new test-sets.

The solution to this issue is $k$-fold cross-validation with the `xgb.cv()` function instead of `.train()`:

In [ ]:
results = xgb.cv(
    params=params
    , dtrain=dtrain_reg
    , num_boost_round=1000
    , verbose_eval=500
    , early_stopping_rounds=20
    , nfold=5
)

[0]	train-rmse:3977.43576+10.07020	test-rmse:3978.85810+40.09560
[63]	train-rmse:402.66435+3.15904	test-rmse:558.10036+9.32226


Instead of passing a test and training set in `evals` in the `.train()` function, we allow the `cv()` function to pseudorandomly create folds with the `nfold` parameter. Instead of a single model object, the `cv()` function returns a list of results that are the mean of cross-validated model results for _every_ boosting round:

In [ ]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,3977.435758,10.070199,3978.858100,40.095596
1,2843.757606,7.006502,2847.097944,31.381355
2,2057.786913,5.627692,2064.569714,25.364942
3,1518.225476,4.319844,1528.032156,22.361016
4,1154.995565,4.154201,1169.602765,17.903318


This means that a new model will need to be trained given the results of CV after the minimum RMSE is found:

In [ ]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

555.694556080344

## XGBoost Classification

The 2 most popular classification objectives are as follows:

- `binary:logistic`: binary classification
- `multi:softprob`: multi-class classification

The technical details of performing these 2 classification methods are nearly identical. For this problem, we will switch up the target variable to be the `'cut'` quality and use price as well as the remaining variables as regressors.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

X, y = diamonds.drop("cut", axis=1), diamonds[['cut']]

#Encode y to be numeric
y_encoded = OrdinalEncoder().fit_transform(y) #"Fair" -> "Good" -> "Very Good" -> "Premium" -> "Ideal"

#Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

#Convert to pd.categorical
for col in cats:
    X[col] = X[col].astype('category')

#Recreate training/test splits w/ y_encoded
X_train, X_test, y_encoded_train, y_encoded_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)

#Create classification matrices
dtrain_clf = xgb.DMatrix(X_train, y_encoded_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_encoded_test, enable_categorical=True)

Note the `enable_categorical` to account for the categorical `y` target. Below, this requires us to specify the number of classes, `"num_class"`.

In [ ]:
params = {
    "objective": "multi:softprob"
    , "tree_method": "hist"
    , "num_class": 5
}

results = xgb.cv(
    params, dtrain_clf
    , num_boost_round=1000
    , nfold=5
    , metrics=["mlogloss", "auc", "merror"]
)

During cross-validation, we are asking XGBoost to monitor 3 different classification metrics: Log-loss, AUC, and whatever the hell `"merror"` is (mean error?).

In [ ]:
max_auc = results['test-auc-mean'].max()
results[results['test-auc-mean'] == max_auc]

,train-mlogloss-mean,train-mlogloss-std,train-auc-mean,train-auc-std,train-merror-mean,train-merror-std,test-mlogloss-mean,test-mlogloss-std,test-auc-mean,test-auc-std,test-merror-mean,test-merror-std
179,0.262976,0.000751,0.989071,0.000088,0.082647,0.000726,0.534053,0.004886,0.940314,0.00081,0.200396,0.003197


## XGBoost SKLearn API

XGBoost offers `XGBClassifier` and `XGBRegressor` classes that easily integrate into a Sci-Kit Learn ecosystem, at a loss of some functionality.

In [ ]:
#Train a model using the sci-kit learn API
xgb_classifier = xgb.XGBClassifier(
    n_estimators=100
    , objective='binary:logistic'
    , tree_method='hist'
    , eta=0.1
    , max_depth=3
    , enable_categorical=True
)
xgb_classifier.fit(X_train, y=y_encoded_train)
print("XGB Classifier fitted!")

XGB Classifier fitted!


This model may be used to `predict` diamond quality outcomes:

In [ ]:
#RMSE
errors = (xgb_classifier.predict(X_test) - y_encoded_test.flatten())
np.sqrt((errors**2).mean())

0.8615289372688394

And, the native XGBoost API model may be extracted with the `.get_booster()` method.

In [ ]:
model = xgb_classifier.get_booster()

## References

- XGBoost Parameters Page: [https://xgboost.readthedocs.io/en/stable/parameter.html](https://xgboost.readthedocs.io/en/stable/parameter.html)
- DataCamp Tutorial: [https://www.datacamp.com/tutorial/xgboost-in-python](https://www.datacamp.com/tutorial/xgboost-in-python)